<a href="https://colab.research.google.com/github/SangHeeRho/bigdataclass_Machine-Learning/blob/main/%EC%8B%A4%EC%8A%B5/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81_%26_%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#네이버 영화 TOP 100 데이터 크롤링하기

##라이브러리 설치

In [ ]:
#링크 수집을 위한 라이브러리 설치
!pip install selenium

##라이브러리 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

##분석할 데이터 모으기(크롤링)

##네이버 영화 TOP 100 페이지 별 링크 수집

In [ ]:
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_D&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_W&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_M&page= 월간


def getPageLinksWantRange(startPageNo, lastPageNo) : 
  links = [] #100개의 영화 링크를 수집할 리스트 변수 선언
  return_links=[]

  for pageNo in range(startPageNo -1, lastPageNo) :
    #주간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_W&page=" + str(pageNo + 1)
    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성/소스코드 통째로 긁어옴
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">


    for movielink in movielinks :
      link = str(movielink.get('href'))
      links.append("https://series.naver.com" + link) # 접속할 수 있는 전체 링크 형태로 변환
  return links


#네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options) #가상의 드라이버 만들기 나 대신 접속해줄
  driver.get(url2)
  time.sleep(3.0) #3초 sleep / 시간을 벌어주는 작업
  
  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') #control t 새창이 열림

  for link in links:
    print(link + ' 수집 중...............')
    driver.switch_to.window(driver.window_handles[-1]) # 새창 활성화
    time.sleep(0.1) #끊길 수 있어서 재움
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    #청소년 관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em ')
      score = float(score.text)
      score = int(score) # 정수형으로 변경
      score_infos.append(score)
      print(score)

      #장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '')
      genre = genre.replace('\n','')
      genre = genre.replace('\t', '')
      genre_infos.append(genre)
      print(genre)

      #제목, 줄거리

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      #영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      #제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)
      print(title)

      #줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0 :
        content_infos.append("줄거리 오류")
      
      else:
        for contents in contents_texts :
          #줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace("\r", '') #줄바꿈 제거
          temp = temp.replace('\xa0','') #공백 제거
          content_infos.append(temp)
      
    elif newflag == '' : #if newflag 와 짝꿍
        print("청불 영화로 데이터 수집하지 않습니다.")

  print("수집 완료 합니다........")
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목":title_infos, "평점":score_infos, "장르":genre_infos, "줄거리":content_infos}

  #딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  #수집된 정보 중에 중복데이터 삭제 (구매/대여)
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') # 줄거리 비교해서, 중복된 영화를 삭제 (첫번째 수집 영화만 남김)

  return movie_df2


##CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_data_' + str(num) + ' .csv'),
                    sep=',' , na_rep='NaN', encoding='euc-kr')
  except:
    print("Error")    

##크롤링 실행

In [ ]:
#1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6291892 수집 중...............
9
애니메이션
극장판 귀멸의 칼날: 무한열차편
https://series.naver.com/movie/detail.nhn?productNo=3400212 수집 중...............
7
공포/스릴러
콰이어트 플레이스
https://series.naver.com/movie/detail.nhn?productNo=2822635 수집 중...............
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3024534 수집 중...............
8
액션
킬러의 보디가드
https://series.naver.com/movie/detail.nhn?productNo=6283368 수집 중...............
6
액션
파이프라인
https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중...............
9
애니메이션
소울
https://series.naver.com/movie/detail.nhn?productNo=3400211 수집 중...............
7
공포/스릴러
콰이어트 플레이스
https://series.naver.com/movie/detail.nhn?productNo=3639460 수집 중...............
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6243813 수집 중...............
9
액션
크루즈 패밀리: 뉴 에이지
https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중...............
7
드라마
미나리
https://series.naver.com/m

#크롤링한 TOP 100 영화 데이터 랭킹 시각화

##라이브러리 임포트

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

##데이터 준비하기

In [3]:
df = pd.read_csv('/content/movie_my_data_realtime_75.csv', encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  75 non-null     int64 
 1   제목          75 non-null     object
 2   평점          75 non-null     int64 
 3   장르          75 non-null     object
 4   줄거리         75 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [4]:
df.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1,1,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...


In [5]:
fig = px.scatter(df, x="제목", y='평점', color='장르')
fig.show()

In [7]:
df_score = df[df.평점 == 9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
60,69,세상에서 가장 아름다운 이별,9,드라마,"“피곤해” 병원 일에만 신경 쓰는 가장(김갑수)“밥 줘, 밥” 어린애가 되어버린 할..."
67,76,말할 수 없는 비밀,9,드라마,나는 너를 사랑해!너도 나를 사랑하니?예술학교로 전학 온 상륜(주걸륜)은 아버지의 ...
68,77,라따뚜이,9,액션,"절대미각, 빠른 손놀림, 끓어 넘치는 열정의 소유자 ‘레미’. 프랑스 최고의 요리사..."
70,79,시간을 달리는 소녀,9,드라마,저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....
73,82,이터널 선샤인,9,드라마,조엘은 아픈 기억만을 지워준다는 라쿠나사를 찾아가 헤어진 연인 클레멘타인의 기억을 ...


###9점 평점 영화 중 장르 차트

In [8]:
fig = px.pie(df_score, values='평점', names='장르')
fig.show()

###일간 TOP 100 시각화

In [10]:
df2 = pd.read_csv('/content/movie_my_data_daily_61.csv', encoding='euc-kr')
df2.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [11]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [12]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

###주간 TOP 100 시각화

In [13]:
df2 = pd.read_csv('/content/movie_my_data_weekly_63.csv', encoding='euc-kr')
df2.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [14]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [15]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

###월간 TOP 100 시각화

In [16]:
df2 = pd.read_csv('/content/movie_my_data_monthly_64.csv', encoding='euc-kr')
df2.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,미나리,7,드라마,"""미나리는 어디서든 잘 자라""낯선 미국, 아칸소로 떠나온 한국 가족.가족들에게 뭔가..."
1,1,노바디,8,액션,비범한 과거를 숨긴 채 남들과 다를 바 없는 평범한 일상을 사는 한 가정의 가장 ‘...
2,2,자산어보,9,드라마,"“이 양반은 대역 죄인이니 너무 잘해줄 생각들 말어”순조 1년, 신유박해로 세상의 ..."
3,3,라야와 마지막 드래곤,9,애니메이션,"인간과 드래곤이 평화롭게 공존하던 신비의 땅, 쿠만드라 왕국살아있는 모든 생명을 삼..."
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [17]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [18]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

###실시간 TOP 100 시각화

In [19]:
df2 = pd.read_csv('/content/movie_my_data_realtime_75.csv', encoding='euc-kr')
df2.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1,1,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...


In [20]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [21]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()